In [2]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency

In [5]:
places = ["Kreuzberg", "Schöneberg"]
gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]

In [7]:
import json
keys = []
for index, row in bounding_boxes.iterrows():
    nodes = get_nodes_with_tags_in_bbox(list(row))
    keys += (list(count_tag_frequency(nodes,).keys()))

unique_keys = set(keys)
unique_keys
   


{'removed:amenity',
 'coffee',
 'self_checkout',
 'image:0',
 'map_size',
 'phone',
 'entrance_marker:subway',
 'regional',
 'identity',
 'removed:inscription',
 'hazard',
 'object:country',
 'service:vehicle:tyres',
 'name:en',
 'park_ride',
 'production_input:knowledge:p2p_licence',
 'planned:natural',
 'visibility',
 'payment:lightning_contactless',
 'disused:amenity',
 'socket:type2_combo:output',
 'seats',
 'indoor_seating',
 'drink:espresso',
 'motorcycle',
 'natural_protection',
 'check_date:internet_access',
 'pub',
 'ref:vatin',
 'country',
 'source:shop',
 'collection_times',
 'removed:highway',
 'name:es',
 'parking',
 'name:ru_1',
 'motorcar',
 'railway:signal:minor:states',
 'seamark:name',
 'operator:phone',
 'disaster_help_point:alert',
 'obstacle_name',
 'reusable_packaging:accept',
 'recycling:green_waste',
 'payment:cryptocurrencies',
 'payment:telephone_cards',
 'railway:signal:main:deactivated',
 'waterway:sign',
 'tomb',
 'railway:signal:distant:height',
 'acc',
 '